# 66degrees Data Engineer Case Study

## 1. Environment Setup

In [26]:
%pip install kaggle pandas sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [27]:
from pathlib import Path
import os, zipfile, sqlite3
import pandas as pd

## Python code for data extraction, transformation, and loading.

In [28]:
RAW_DIR   = Path("data/raw")
PROCESSED = Path("data/processed")
DB_FILE   = "supermarket.db"

def download_from_kaggle():
    """
    Downloads the dataset via Kaggle API.
    """
    RAW_DIR.mkdir(parents=True, exist_ok=True)
    os.system("kaggle datasets download -d lovishbansal123/sales-of-a-supermarket -p data/raw")
    
    zip_path = next(RAW_DIR.glob("*.zip"))
    if not zip_path:
        raise FileNotFoundError("ZIP not found. Download may have failed.")
    with zipfile.ZipFile(zip_path) as zf:
        zf.extractall(RAW_DIR)
    print("Downloaded and extract complete")


## Dim tables

In [33]:
def build_dimensions(df: pd.DataFrame):
    # Rename columns up front to consistent internal names
    df = df.rename(columns={
        "Product line": "product_line",
        "Unit price": "unit_price",
        "Branch": "branch",
        "City": "city"
    })

    # Build product dimension
    dim_product = (
        df[["product_line", "unit_price"]]
        .drop_duplicates()
        .reset_index(drop=True)
    )
    dim_product["product_id"] = dim_product.index + 1

    # Build store dimension
    dim_store = (
        df[["branch", "city"]]
        .drop_duplicates()
        .reset_index(drop=True)
    )
    dim_store["store_id"] = dim_store.index + 1

    return dim_product, dim_store


## Fact table

In [34]:
def build_fact(df, dim_product, dim_store):
    # Rename columns to match merge keys
    df = df.rename(columns={
        "Product line": "product_line",
        "Unit price": "unit_price",
        "Branch": "branch",
        "City": "city"
    })

    fact = (
        df.merge(dim_product, on=["product_line", "unit_price"])
          .merge(dim_store, on=["branch", "city"])
          .assign(sale_id=lambda d: d.index + 1)
          .rename(columns={
              "Date": "date",
              "Customer type": "customer_type",
              "Gender": "gender",
              "Total": "total",
              "Quantity": "quantity",
              "Payment": "payment"
          })
    )

    fact_cols = ["sale_id", "date", "customer_type", "gender",
                 "product_id", "store_id", "total", "quantity", "payment"]
    return fact[fact_cols]


In [35]:
def write_to_sqlite(dim_product, dim_store, fact_sales):
    conn = sqlite3.connect(DB_FILE)
    dim_product.to_sql("dim_product", conn, index=False, if_exists="replace")
    dim_store.to_sql("dim_store", conn, index=False, if_exists="replace")
    fact_sales.to_sql("fact_sales", conn, index=False, if_exists="replace")
    conn.close()
    print(f"SQLite DB created: {DB_FILE}")


In [36]:
if __name__ == "__main__":
    download_from_kaggle()
    df = pd.read_csv(RAW_DIR / "supermarket_sales.csv")
    dim_product, dim_store = build_dimensions(df)
    fact_sales = build_fact(df, dim_product, dim_store)
    write_to_sqlite(dim_product, dim_store, fact_sales)
    

Dataset URL: https://www.kaggle.com/datasets/lovishbansal123/sales-of-a-supermarket
License(s): apache-2.0
sales-of-a-supermarket.zip: Skipping, found more recently modified local copy (use --force to force download)
Downloaded and extract complete
SQLite DB created: supermarket.db


In [40]:
# takes a SQL query as an argument and returns a pandas dataframe of that query.
def run_query(q):
    with sqlite3.connect(DB_FILE) as conn:
        return pd.read_sql(q, conn)


## Helper functions

In [41]:
#  call the run_query() function to return a list of all tables and views in the database.        
def show_tables():
    q = "SELECT name, type FROM sqlite_master WHERE type IN ('table', 'view')"
    return run_query(q)

In [42]:
show_tables()

,name,type
0,dim_product,table
1,dim_store,table
2,fact_sales,table


## Count rows

## Table row counts

In [43]:
q = """
SELECT 'dim_product' AS table_name, COUNT(*) AS row_count FROM dim_product
UNION
SELECT 'dim_store', COUNT(*) FROM dim_store
UNION
SELECT 'fact_sales', COUNT(*) FROM fact_sales;
"""
run_query(q)

,table_name,row_count
0,dim_product,993
1,dim_store,3
2,fact_sales,1000


In [ ]:
%%sql
SELECT * FROM fact_sales

## Tables Schema.

In [45]:
tables = ["dim_product", "dim_store", "fact_sales"]

for table in tables:
    print(f"Schema for table: {table}")
    display(run_query(f"PRAGMA table_info('{table}')"))

Schema for table: dim_product


,cid,name,type,notnull,dflt_value,pk
0,0,product_line,TEXT,0,None,0
1,1,unit_price,REAL,0,None,0
2,2,product_id,INTEGER,0,None,0


Schema for table: dim_store


,cid,name,type,notnull,dflt_value,pk
0,0,branch,TEXT,0,None,0
1,1,city,TEXT,0,None,0
2,2,store_id,INTEGER,0,None,0


Schema for table: fact_sales


,cid,name,type,notnull,dflt_value,pk
0,0,sale_id,INTEGER,0,None,0
1,1,date,TEXT,0,None,0
2,2,customer_type,TEXT,0,None,0
3,3,gender,TEXT,0,None,0
4,4,product_id,INTEGER,0,None,0
5,5,store_id,INTEGER,0,None,0
6,6,total,REAL,0,None,0
7,7,quantity,INTEGER,0,None,0
8,8,payment,TEXT,0,None,0


In [46]:

q = """
-- Column null check
SELECT 
  COUNT(*) AS null_customer_type 
FROM fact_sales 
WHERE customer_type IS NULL;
"""
run_query(q)


,null_customer_type
0,0


In [47]:
q= """
SELECT 
  s.date,
  p.product_line,
  SUM(s.total) AS total_sales,
  RANK() OVER (PARTITION BY s.date ORDER BY SUM(s.total) DESC) AS rank_by_day
FROM fact_sales s
JOIN dim_product p ON s.product_id = p.product_id
GROUP BY s.date, p.product_line
ORDER BY s.date, total_sales DESC;"""
run_query(q)

,date,product_line,total_sales,rank_by_day
0,1/1/2019,Electronic accessories,1382.7450,1
1,1/1/2019,Sports and travel,1143.3870,2
2,1/1/2019,Food and beverages,1066.0230,3
3,1/1/2019,Fashion accessories,621.2430,4
4,1/1/2019,Home and lifestyle,399.7560,5
...,...,...,...,...
449,3/9/2019,Sports and travel,1877.9565,2
450,3/9/2019,Food and beverages,1459.1115,3
451,3/9/2019,Electronic accessories,980.0070,4
452,3/9/2019,Health and beauty,762.5520,5


In [48]:
q = """
-- Product sales totals
SELECT 
  p.product_line,
  COUNT(*) AS num_sales,
  SUM(s.total) AS total_revenue
FROM fact_sales s
JOIN dim_product p ON p.product_id = s.product_id
GROUP BY p.product_line
ORDER BY total_revenue DESC;"""

run_query(q)

,product_line,num_sales,total_revenue
0,Food and beverages,174,56144.8440
1,Sports and travel,166,55122.8265
2,Electronic accessories,170,54337.5315
3,Fashion accessories,178,54305.8950
4,Home and lifestyle,160,53861.9130
5,Health and beauty,152,49193.7390


In [49]:
q = """
-- Store sales summary
SELECT 
  d.city,
  d.branch,
  SUM(f.total) AS revenue,
  COUNT(*) AS transactions
FROM fact_sales f
JOIN dim_store d ON d.store_id = f.store_id
GROUP BY d.city, d.branch
ORDER BY revenue DESC;"""
run_query(q)

,city,branch,revenue,transactions
0,Naypyitaw,C,110568.7065,328
1,Yangon,A,106200.3705,340
2,Mandalay,B,106197.6720,332
